In [2]:
projDirectory = '/Users/sean/mbp_storage/251019/'
sails = ['portMain', 'stbMain', 'portJib', 'stbJib']
mains = [0,1]
jibs = [2,3]
scans = []
for i in mains:
    scans.append(pd.read_csv(os.path.join(projDirectory,sails[i],'autScan_output','data.csv')))

event_df = pd.read_csv(os.path.join(projDirectory,'event','event_clean.csv'))
log_df = pd.read_csv(os.path.join(projDirectory,'log','log_clean.csv'))

In [3]:
scan_df = pd.concat(scans)


In [4]:
tws_bin_edges = [0,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5,12.5,15,17,19,21,23,25,27,30]
tws_bin_labels = ['<4', '4', '5', '6', '7', '8', '9', '10', '11', '12', '14', '16', '18', '20', '22', '24', '26', '>26']
forestay_bin_edges = [0,0.3,0.5,0.7,0.9,1.1,1.3,1.5,1.7,1.9,2.1,2.3,2.5]
forestay_bin_labels = ['<0.4', '0.4', '0.6', '0.8', '1.0', '1.2', '1.4', '1.6', '1.8', '2.0', '2.2', '2.4']

In [5]:
data_df = pd.merge(left=scan_df, right=event_df, how='left', on='timeStamp')

data_df = pd.merge(left=data_df, right=log_df, how='left', on='timeStamp')
data_df['TWS_bin'] = pd.cut(data_df['TWS'], bins = tws_bin_edges, labels= tws_bin_labels)
data_df['foreStay_bin'] = pd.cut(data_df['Forestay'], bins = forestay_bin_edges, labels= forestay_bin_labels)
toMultiply = ['draft', 'camber']
for var in toMultiply:
    data_df[var] = data_df[var]*100

In [8]:
ind = ['timeStamp', 'Height']
agregation_vars = ['camber', 'draft', 'frontCamber', 'backCamber', 'twist', 'TWS']
vertical_vars = ['foreStay_bin']
horizontal_vars = ['Race']
horizontal_dfs = []
for i in range(len(horizontal_vars)):
    filter = data_df[horizontal_vars[i]].unique()
    filter.sort()
    for j in range(len(filter)):
        agregation_vars.insert(0, horizontal_vars[i])
        df = data_df[data_df[horizontal_vars[i]] == filter[j]]
        col_list = ind + agregation_vars+vertical_vars
        groupVars = vertical_vars+['Height']
        df = df[col_list]
        df = df.groupby(groupVars, observed=True)[agregation_vars].mean()
        horizontal_dfs.append(df)
        agregation_vars.remove(horizontal_vars[i])


In [9]:
horizontal_dfs[0]

Race     camber      draft  frontCamber  backCamber  \
foreStay_bin Height                                                        
0.8          25       1.0   9.434024  45.245126     0.744487    0.703914   
             50       1.0  11.664725  44.135451     0.778467    0.693045   
             75       1.0  10.529315  42.486716     0.779261    0.665599   
             87       1.0   6.619760  38.951420     0.784304    0.604318   
1.0          25       1.0   9.233591  45.600327     0.742117    0.703422   
             50       1.0  11.444644  44.533209     0.777368    0.696405   
             75       1.0  10.147070  42.785878     0.767759    0.672122   
             87       1.0   6.531559  40.072111     0.794921    0.607772   
1.2          25       1.0   8.886862  45.985383     0.738644    0.705574   
             50       1.0  10.985776  45.203668     0.773195    0.703182   
             75       1.0   8.861888  43.673251          inf         inf   
             87       1.0   6.311871  40.295723     0.785884    0.624523   
1.4          25       1.0   8.654091  45.926605     0.733974    0.698856   
             50       1.0  10.660352  45.364963     0.769155    0.700979   
             75       1.0   8.936722  43.758516     0.776856    0.678816   
             87       1.0   5.709706  40.927789     0.787337    0.630193   

                         twist       TWS  
foreStay_bin Height                       
0.8          25       3.805030  8.478125  
             50       6.242573  8.478125  
             75       9.164970  8.478125  
             87      10.002472  8.478125  
1.0          25       3.230390  8.057719  
             50       5.629084  8.057719  
             75       8.006317  8.057719  
             87       8.786602  8.057719  
1.2          25       3.568410  8.474000  
             50       6.251276  8.474000  
             75       7.176027  8.474000  
             87      10.135632  8.474000  
1.4          25       3.522098  8.897407  
             50       6.759330  8.897407  
             75       8.947522  8.897407  
             87      11.000318  8.897407

In [10]:
horizontal_dfs[1]

Race     camber      draft  frontCamber  backCamber  \
foreStay_bin Height                                                        
1.0          25       2.0   8.891985  45.858975     0.740615    0.706338   
             50       2.0  11.245019  45.177635     0.771339    0.701936   
             75       2.0   9.901536  42.951392     0.776440    0.674040   
             87       2.0   6.611065  39.908955     0.786015    0.619877   
1.2          25       2.0   8.741640  46.490277     0.726588    0.701159   
             50       2.0  10.641987  46.052814         -inf    0.704835   
             75       2.0   9.225066  44.148180     0.765728    0.683088   
             87       2.0   5.988609  40.162429     0.781501    0.630892   
1.4          25       2.0   8.390731  47.072993     0.717652    0.699331   
             50       2.0  10.118905  47.113213     0.729927    0.711560   
             75       2.0   8.108837  45.080714          inf         inf   
             87       2.0   5.490185  41.316040     0.783137    0.641585   
1.6          25       2.0   8.335029  47.373659     0.714540    0.696052   
             50       2.0   9.943467  48.008524     0.738128    0.706842   
             75       2.0   6.694413  45.877267     0.606437    0.362367   
             87       2.0   5.240323  42.629208     0.781290    0.658239   

                         twist        TWS  
foreStay_bin Height                        
1.0          25       3.814194   8.808000  
             50       6.639153   8.808000  
             75       9.626546   8.808000  
             87      10.235821   8.808000  
1.2          25       3.971502   9.356591  
             50       7.109295   9.356591  
             75      10.220828   9.356591  
             87      11.374354   9.356591  
1.4          25       4.191702  10.192222  
             50       7.777350  10.192222  
             75      10.659960  10.192222  
             87      12.554662  10.192222  
1.6          25       4.088006  10.160588  
             50       8.198303  10.160588  
             75       9.950608  10.160588  
             87      13.042939  10.160588

In [11]:
horizontal_dfs[2]

Race     camber      draft  frontCamber  backCamber  \
foreStay_bin Height                                                        
0.8          25       3.0  10.080255  43.997210     0.737422    0.677167   
             50       3.0  12.006970  43.425438     0.772134    0.683599   
             75       3.0  10.288855  42.455170     0.769633    0.668439   
             87       3.0   6.457853  41.039323     0.779922    0.640226   
1.0          25       3.0   9.496182  45.001330     0.735029    0.686683   
             50       3.0  11.156125  44.733379     0.765958    0.695063   
             75       3.0   9.636646  42.839385     0.776772    0.668574   
             87       3.0   6.067334  40.977622     0.786701    0.632851   
1.2          25       3.0   8.735246  45.882029     0.732283    0.697081   
             50       3.0  10.764166  45.808252     0.757119    0.702924   
             75       3.0   9.149779  43.751973     0.770756    0.687637   
             87       3.0   5.431743  40.498110     0.781077    0.635850   
1.4          25       3.0   8.652460  47.014936     0.713778    0.697736   
             50       3.0  10.280028  47.087180     0.742247    0.704069   
             75       3.0   8.809986  44.807580     0.764625    0.697013   
             87       3.0   5.391012  41.297828     0.784216    0.638099   
1.6          25       3.0   8.357770  48.046385     0.701351    0.700827   
             50       3.0  10.075381  48.423178     0.725621    0.706716   
             75       3.0   8.296530  46.065028     0.756901    0.705155   
             87       3.0   4.760133  41.615349     0.793745    0.629568   

                         twist        TWS  
foreStay_bin Height                        
0.8          25       3.540999   8.791000  
             50       7.820393   8.791000  
             75      12.023300   8.791000  
             87      12.947691   8.791000  
1.0          25       3.406752   8.792667  
             50       7.320136   8.792667  
             75      10.461011   8.792667  
             87       9.934625   8.792667  
1.2          25       3.904596   9.476667  
             50       7.671224   9.476667  
             75      11.360404   9.476667  
             87      12.373326   9.476667  
1.4          25       4.433548  10.615128  
             50       8.044998  10.615128  
             75      12.044412  10.615128  
             87      13.202293  10.615128  
1.6          25       4.906191  10.780000  
             50       8.944372  10.780000  
             75      12.840285  10.780000  
             87      14.170071  10.780000